In [2]:
#!/usr/local/bin/python

# This version of the code trains the attractor connections with a separate
# objective function than the objective function used to train all other weights
# in the network (on the prediction task).

from __future__ import print_function
import itertools
import tensorflow as tf
import numpy as np
import sys
import argparse
import datetime


% load_ext autoreload
% autoreload

from tensorflow_helpers import *
from data_generator import generate_examples, pick_task

from helper_functions import get_batches, load_pretrained_embeddings, \
    get_model_type_str, translate_ids_to_words, \
    save_results, print_into_log, print_some_translated_sentences, \
    get_training_progress_comment
from information_trackers import compute_entropy_fullvec
from graph_init import GRU_attractor


class EarlyStopper():
    def __init__(self, patience_max, disp_epoch, min_delta = 0.00):
        self.best = 1e10
        self.patience = 0  # our patience
        self.patience_max = patience_max
        self.display_epoch = disp_epoch
        self.min_delta = min_delta

    def update(self, current):
        if self.best > current:
            self.best = current
            self.patience = 0
        elif abs(self.best - current) > self.min_delta:
            self.patience += 1

    def patience_ran_out(self):
        if self.patience*self.display_epoch > self.patience_max:
            return True
        else:
            False
            
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference


ops = {
    'model_type': "GRU",  # OPTIONS: vanilla, LSTM_raw, LSTM_tensorflow, LSTM_attractor
    'hid': 50,
    'in': None,  # TBD
    'out': 1,
    #         'batch_size':n_examples, #since the sequences are 1-dimensional it's easier to just run them all at once
    'n_attractor_iterations': 15,
    'attractor_dynamics': "projection2",  # OPTIONS:  "" (for no attractor dynamics),
    #           "direct" (simple attractor weights applied to hidden states directly, trained with noise addition)
    #           "projection" (project the hidden state into a separate space via weights, do attraction, project back)
    #           "helper_hidden" (hidden-hidden neurons) - IMPORTANT: don't forget to add h_hid number
    'h_hid': 100,  # helper hidden for "helper hidden" "attractory_dynamics" mode
    'attractor_noise_level': 0.5,
    'attractor_noise_type': "bernoilli",  # OPTIONS: "gaussian", "dropout", "random_drop"

    'training_mode': "",  # 'attractor_on_both',

    'attractor_regularization': "l2_regularization",  # OPTIONS: "l2_regularization", "l2_norm"
    'attractor_regularization_lambda': 0.0,

    'record_mutual_information': True,
    'problem_type': "msnbc",  # OPTIONS: parity, parity_length, majority, reber, kazakov, pos_brown, ner_german, sentiment_imdb, topic_classification, video_classification
    'masking': True,#"seq", "final"
    'prediction_type': 'seq', #'seq', 'final', 'final_class'
    'seq_len': None,

    'save_best_model': True,
    'reshuffle_data_each_replication': False,  # relevant for POS datasets (since they are loaded from files)
    'test_partition': 0.3,
    'lrate': 0.001,  # was 0.008

    # NLP related (pos_brown task)
    'bidirectional': False,
    'embedding_size': 100,
    'load_word_embeddings': False,
    'train_word_embeddings': False,
    'trainable_logic_symbols': 0, #make first *N* embeddings trainable(Pad, unknown, start symbols make it a separate matrix and trainable)
    'input_type': "embed",  # embed&prior, embed, prior
    'dropout': 0.2  # in range(0,1)
}

# !!!!!!!!!!!!!!!!!!!!!!
# SEQ_LEN = 12 # number of bits in input sequence
N_HIDDEN = ops['hid']  # number of hidden units
N_H_HIDDEN = ops['h_hid']
TASK = ops['problem_type']
ARCH = ops['model_type']  # hidden layer type: 'GRU' or 'tanh'
NOISE_LEVEL = ops['attractor_noise_level']
# noise in training attractor net
# if >=0, Gaussian with std dev NOISE_LEVEL
# if < 0, Bernoulli dropout proportion -NOISE_LEVEL

# !!!!!!!!!!!!!!!!!!!!!!
INPUT_NOISE_LEVEL = 0.1
ATTRACTOR_TYPE = ops['attractor_dynamics']
N_ATTRACTOR_STEPS = ops['n_attractor_iterations']
# number of time steps in attractor dynamics
# if = 0, then no attractor net
# !!!!!!!!!!!!!!!!!!!!!!
# ATTR_WEIGHT_CONSTRAINTS = True
# True: make attractor weights symmetric and have zero diag
# False: unconstrained
TRAIN_ATTR_WEIGHTS_ON_PREDICTION = False
# True: train attractor weights on attractor net _and_ prediction
REPORT_BEST_TRAIN_PERFORMANCE = True
# True: save the train/test perf on the epoch for which train perf was best
LOSS_SWITCH_FREQ = 1
# how often (in epochs) to switch between attractor
# and prediction loss

ops, SEQ_LEN, N_INPUT, N_CLASSES, N_TRAIN, N_TEST = pick_task(ops['problem_type'],
                                                              ops)  # task (parity, majority, reber, kazakov)

# Training Parameters

TRAINING_EPOCHS = 500
N_REPLICATIONS = 1
BATCH_SIZE = 8000
DISPLAY_EPOCH = 1
EARLY_STOPPING_THRESH = 0.03 # 1e-3 for POS, 0.03 for Sentiment
EARLY_STOPPING_PATIENCE = 20  # in epochs
EARLY_STOPPING_MINIMUM_EPOCH = 0

# NOTEBOOK CODE

######### MAIN CODE #############################################################
#0.02, 0.05, 0.1, 0.2, 0.35, 0.5, 
for dataset_part in [0.5, 0.99]:
#     for attractor_steps in [15,0]:
    attractor_steps = 15
    for attractor_steps in [15,0]:
        NOISE_LEVEL = ops['attractor_noise_level']


#     for att_reg in [0.0]:
#         ops['attractor_regularization_lambda'] = att_reg
        # the tf seed needs to be within the context of the graph.
        tf.reset_default_graph()
        np.random.seed(11)
        tf.set_random_seed(11)
        ops['n_attractor_iterations'] = attractor_steps
        N_ATTRACTOR_STEPS = ops['n_attractor_iterations']

        #
        # PLACEHOLDERS
        #
        if 'pos' in ops['problem_type']:
            # X will be looked up in the embedding table, so the last dimension is just a number
            X = tf.placeholder("int64", [None, SEQ_LEN], name='X')
            # last dimension is left singular, tensorflow will expect it to be an id number, not 1-hot embed
            Y = tf.placeholder("int64", [None, SEQ_LEN], name='Y')
        elif 'msnbc' in ops['problem_type']:
            X = tf.placeholder("int64", [None, SEQ_LEN], name='X')
            Y = tf.placeholder("int64", [None, SEQ_LEN], name='Y')
        elif ops['problem_type'] == 'sentiment_imdb':
             # X will be looked up in the embedding table, so the last dimension is just a number
            X = tf.placeholder("int64", [None, SEQ_LEN], name='X')
            Y = tf.placeholder("int64", [None, N_CLASSES], name='Y')
        elif ops['problem_type'] == 'topic_classification':
             # X will be looked up in the embedding table, so the last dimension is just a number
            X = tf.placeholder("int64", [None, SEQ_LEN], name='X')
            Y = tf.placeholder("int64", [None, 1], name='Y')
        elif ops['problem_type'] == 'ner_german':
            X = tf.placeholder("float", [None, SEQ_LEN, N_INPUT])
            Y = tf.placeholder("int64", [None, SEQ_LEN])
        else:  # single output 
            X = tf.placeholder("float", [None, SEQ_LEN, N_INPUT])
            Y = tf.placeholder("int64", [None, 1])
        attractor_tgt_net = tf.placeholder("float", [None, N_HIDDEN], name='attractor_tgt')

        # Embedding matrix initialization
        if 'pos' in ops['problem_type'] or 'sentiment' in ops['problem_type'] or ops['problem_type'] == "topic_classification":
            [_, _, _, _, _, _, maps] = generate_examples(SEQ_LEN, N_TRAIN, N_TEST,
                                                         INPUT_NOISE_LEVEL, TASK, ops)

            if ops['load_word_embeddings']:
                embeddings_loaded, _ = load_pretrained_embeddings('data/glove.6B.{}d.txt'.format(ops['embedding_size']),
                                                               maps, ops)
                if ops['trainable_logic_symbols'] > 0:
                    with tf.variable_scope("TASK_WEIGHTS"):
                        symbols_embedding = tf.get_variable("symb_embedding",
                                                initializer=tf.truncated_normal_initializer(stddev=0.05),
                                                shape=[ops['trainable_logic_symbols'], ops['embedding_size']],
                                                dtype=tf.float32,
                                                trainable=True)
                    
                word_embedding = tf.get_variable("embedding",
                                            initializer=embeddings_loaded,
                                            dtype=tf.float32,
                                            trainable=ops['train_word_embeddings'])
                if ops['trainable_logic_symbols'] > 0:
                    embedding = tf.concat([symbols_embedding, word_embedding], axis=0)
                else:
                    embedding = word_embedding
            else:  # initialize randomly
                embedding = tf.get_variable("embedding",
                                            initializer=tf.truncated_normal_initializer(stddev=0.05),
                                            shape=[ops['vocab_size'], ops['embedding_size']],
                                            dtype=tf.float32,
                                            trainable=ops['train_word_embeddings'])
            embed_lookup = tf.nn.embedding_lookup(embedding, X)

            # load priors information
            if ops['input_type'] == 'prior' or ops['input_type'] == 'embed&prior':
                id2prior = maps['id2prior']
                word2id = maps['word2id']
                priors = np.zeros([len(id2prior), len(id2prior[0])]).astype("float32")
                for id, prior in id2prior.items():
                    priors[id] = prior
                priors_op = tf.get_variable("priors",
                                            initializer=priors,
                                            dtype=tf.float32,
                                            trainable=False)
                prior_lookup = tf.nn.embedding_lookup(priors_op, X)

            if ops['input_type'] == 'embed':
                embed = embed_lookup
            elif ops['input_type'] == 'prior':
                embed = prior_lookup
            elif ops['input_type'] == 'embed&prior':
                embed = tf.concat([embed_lookup, prior_lookup], axis=2)

        # Graph + all the training variables
        if 'pos' in ops['problem_type']:
            net_inputs = {'X': embed, 'mask': Y, 'attractor_tgt_net': attractor_tgt_net}
        elif ops['problem_type'] == 'sentiment_imdb' or ops['problem_type'] == 'topic_classification':
            net_inputs = {'X': embed, 'mask': X, 'attractor_tgt_net': attractor_tgt_net}
        elif ops['problem_type'] == 'msnbc':
            net_inputs = {'X': tf.one_hot(X, depth=N_CLASSES), 'mask': X, 'attractor_tgt_net': attractor_tgt_net}
        else:
            net_inputs = {'X': X, 'mask': X, 'attractor_tgt_net': attractor_tgt_net}

        if ops['bidirectional']:
            G_attractors = {'forw': [], 'back': []}
            names = G_attractors.keys()
            # Forward:
            G_forw = GRU_attractor(ops, inputs=net_inputs, direction='forward', suffix=names[0])
            attr_loss_op_forw = G_forw.attr_loss_op
            attr_train_op_forw = G_forw.attr_train_op
            h_clean_seq_flat_forw = G_forw.h_clean_seq_flat  # for computing entropy of states
            h_net_seq_flat_forw = G_forw.h_net_seq_flat  # -> attractor_tgt_net placeholder
            G_attractors['forw'] = {'attr_loss_op': attr_loss_op_forw, "attr_train_op": attr_train_op_forw,
                                    'h_clean_seq_flat': h_clean_seq_flat_forw, 'h_net_seq_flat': h_net_seq_flat_forw}
            G_forw_output = G_forw.output

            # Backward:
            G_back = GRU_attractor(ops, inputs=net_inputs, direction='backward', suffix=names[1])
            attr_loss_op_back = G_back.attr_loss_op
            attr_train_op_back = G_back.attr_train_op
            h_clean_seq_flat_back = G_back.h_clean_seq_flat  # for computing entropy of states
            h_net_seq_flat_back = G_back.h_net_seq_flat  # -> attractor_tgt_net placeholder
            G_attractors['back'] = {'attr_loss_op': attr_loss_op_back, "attr_train_op": attr_train_op_back,
                                    'h_clean_seq_flat': h_clean_seq_flat_back, 'h_net_seq_flat': h_net_seq_flat_back}
            G_back_output = G_back.output

            
            
            # Merge: [seq_len, batch_size, n_hid*2]
            # Note that we reverse the backward cell's output to align with original direction
            # note in "final" only prediction, one less dimension
            if 'final' in ops['prediction_type']:
                merge_index = 1
            else:
                merge_index = 2
            output = tf.concat([G_forw_output, tf.reverse(G_back_output, axis=[0])], axis=merge_index)
    
            if ops['dropout'] > 0.0:
                # note keep_prob = 1.0 - drop_probability (not sure why they implemented it this way)
                # tensorflow implementation scales by 1/keep_prob automatically
                output = tf.nn.dropout(output, keep_prob=1.0 - ops['dropout'])
            else:
                output = output

            input_size_final_projection = 2 * ops['hid']
            Y_ =  project_into_output(Y, output, input_size_final_projection, ops['out'], ops)
            
            # LOSS, ACC, & TRAIN OPS
            pred_loss_op = task_loss(Y, Y_, ops)
            optimizer_pred = tf.train.AdamOptimizer(learning_rate=0.008)
            prediction_parameters = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "TASK_WEIGHTS")
            pred_train_op = optimizer_pred.minimize(pred_loss_op, var_list=prediction_parameters)
            accuracy = task_accuracy(Y, Y_, ops)
        else:
            G_attractors = {'forw': []}
            names = G_attractors.keys()
            # Forward:
            G_forw = GRU_attractor(ops, inputs=net_inputs, direction='forward', suffix=names[0])
            attr_loss_op_forw = G_forw.attr_loss_op
            attr_train_op_forw = G_forw.attr_train_op
            h_clean_seq_flat_forw = G_forw.h_clean_seq_flat  # for computing entropy of states
            h_net_seq_flat_forw = G_forw.h_net_seq_flat  # -> attractor_tgt_net placeholder
            G_attractors['forw'] = {'attr_loss_op': attr_loss_op_forw, "attr_train_op": attr_train_op_forw,
                                    'h_clean_seq_flat': h_clean_seq_flat_forw, 'h_net_seq_flat': h_net_seq_flat_forw}
            G_forw_output = G_forw.output

            input_size_final_projection = ops['hid']
            
            if ops['dropout'] > 0.0:
                output = tf.nn.dropout(G_forw_output, keep_prob=1.0 - ops['dropout'])
            else:
                output = G_forw_output
            
            Y_ = project_into_output(Y, output, input_size_final_projection, ops['out'], ops)

            # LOSS, ACC, & TRAIN OPS
            pred_loss_op = task_loss(Y, Y_, ops)
            optimizer_pred = tf.train.AdamOptimizer(learning_rate=0.008)
            prediction_parameters = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "TASK_WEIGHTS")
            pred_train_op = optimizer_pred.minimize(pred_loss_op, var_list=prediction_parameters)
            accuracy = task_accuracy(Y, Y_, ops)

            
        mask_op = tf.cast(tf.sign(Y), dtype=tf.float32)
        # Initialize the variables (i.e. assign their default value)
        init = tf.global_variables_initializer()
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.9)
        with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
            # TODO: make a class for all "best" quantities (a lot of space)
            saved_train_acc = []
            saved_test_acc = []
            saved_epoch = []
            saved_att_loss = []
            saved_entropy_final = []
            saved_val_acc = []
            saved_val_loss = []
            saved_traini_loss = []
            saver = tf.train.Saver()

            # Start training
            for replication in range(N_REPLICATIONS):
                print("********** replication ", replication, " **********")
                early_stopper = EarlyStopper(EARLY_STOPPING_PATIENCE, DISPLAY_EPOCH)
                [X_full_train, Y_full_train, X_test, Y_test, X_val, Y_val, maps] = generate_examples(SEQ_LEN, N_TRAIN, N_TEST,
                                                                                           INPUT_NOISE_LEVEL, TASK, ops)
                # Take Only part of dataset:
                all_ids = range(len(X_full_train))
                np.random.shuffle(all_ids)
                train_part = int(dataset_part * len(X_full_train))
                ids_to_take = all_ids[0:train_part]
                ids_for_val = all_ids[train_part:int(train_part + 0.2*train_part)]
                if len(ids_to_take) > X_full_train.shape[0]:
                    ids_to_take = range(X_full_train.shape[0])
                X_train = X_full_train[ids_to_take]
                Y_train = Y_full_train[ids_to_take]
                
                if BATCH_SIZE < len(X_train):
                    ops['attractor_regularization_lambda'] = ops['attractor_regularization_lambda']/(len(X_train)*1.0/BATCH_SIZE)
                    print(ops['attractor_regularization_lambda'])
                
                X_val, Y_val = X_full_train[ids_for_val], Y_full_train[ids_for_val,:]
                
                N_TRAIN = len(X_train)
                print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

                # Log Path init-n:
                COMMENT = 'dataset_starvation_experiment'
                MODEL_NAME_FILE = '{}_(att_iter{}__bidir{}__drop{})_{}.txt'.format(ops['problem_type'],
                                                                                   ops['n_attractor_iterations'],
                                                                                   ops['bidirectional'],
                                                                                   ops['dropout'],
                                                                                   COMMENT)
                LOG_DIRECTORY = 'experiments/logs/{}'.format(MODEL_NAME_FILE)
                MODEL_DIRECTORY = 'experiments/logs/{}_{}'.format(datetime.date.today(), MODEL_NAME_FILE)
                print_into_log(LOG_DIRECTORY, get_model_type_str(ops, N_TRAIN, N_TEST, SEQ_LEN))
                print_into_log(MODEL_DIRECTORY, get_model_type_str(ops, N_TRAIN, N_TEST, SEQ_LEN), supress=True)

                sess.run(init)  # Run the initializer

                train_prediction_loss = True
                best_train_acc = -1000.
                best_test_acc = 0
                best_entropy = 0.0
                best_att_loss = 0
                best_train_loss = 0
                best_val_loss = 0.0
                best_val_acc = 0.0
                best_epoch = 0
                for epoch in range(1, TRAINING_EPOCHS + 2):
                    if (epoch - 1) % DISPLAY_EPOCH == 0:
                        # TRAIN set:
                        ploss, train_acc = batch_tensor_collect(sess, [pred_loss_op, accuracy],
                                                                X, Y, X_train, Y_train, BATCH_SIZE)
                        # TEST set:
                        test_acc = batch_tensor_collect(sess, [accuracy], X, Y, X_test, Y_test, BATCH_SIZE)[0]
    
                        # Validation set & Early stopping:
                        ploss_val, val_acc = batch_tensor_collect(sess, [pred_loss_op, accuracy],
                                                                  X, Y, X_val, Y_val, BATCH_SIZE)
            
                        # Precistion/Recall:
                        if ops['problem_type'] == 'ner_german':
                            y_pred, y_true, mask_val = batch_tensor_collect(sess, [Y_, Y, mask_op],
                                                                X, Y, X_test, Y_test, BATCH_SIZE)
                            y_pred = np.argmax(y_pred, axis=2)
                            
                            Y_pred_flat = np.extract(mask_val.astype(bool), y_pred)
                            Y_test_flat = np.extract(mask_val.astype(bool), y_true)
                            print("PRECISION:",compute_f1(Y_pred_flat, Y_test_flat, maps['id2tag']))
                            
                        print(early_stopper.patience, early_stopper.best, ploss_val)
                        early_stopper.update(ploss_val)
                        if early_stopper.patience_ran_out():
                            print_into_log(LOG_DIRECTORY, "STOPPED EARLY AT {}".format(epoch))
                            break

                        # ATTRACTOR(s) LOSS
                        aloss = {}
                        entropy = {}
                        hid_vals_arr = batch_tensor_collect(sess, [A['h_net_seq_flat'] for att_name, A in
                                                                   G_attractors.items()],
                                                            X, Y, X_train, Y_train, BATCH_SIZE)
                        h_clean_val_arr = batch_tensor_collect(sess, [A['h_clean_seq_flat'] for att_name, A in
                                                                      G_attractors.items()],
                                                               X, Y, X_train, Y_train, BATCH_SIZE)
                        for i, attractor_name in enumerate(G_attractors.keys()):
                            A = G_attractors[attractor_name]
                            a_loss_val = []
                            n_splits = np.max([1, int(len(X_train) / BATCH_SIZE)])
                            for batch_hid_vals in np.array_split(hid_vals_arr[i], n_splits):
                                a_loss_val.append(
                                    sess.run(A['attr_loss_op'], feed_dict={attractor_tgt_net: batch_hid_vals}))
                            aloss[attractor_name] = "{:.4f}".format(np.mean(a_loss_val))

                            entropy[attractor_name] = "{:.4f}".format(
                                compute_entropy_fullvec(h_clean_val_arr[i], ops, n_bins=8))

                        # Print training information:
                        print_into_log(LOG_DIRECTORY, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + get_training_progress_comment(epoch, ploss, aloss, ploss_val, val_acc, train_acc,
                                                                     test_acc, entropy))
                        # Update the logs:
                       
                        #                 if ops['record_mutual_information']:
                        # #                     h_attractor_val, h_clean_val = sess.run([h_attractor_collection, h_clean_seq_flat],
                        # #                                                                    feed_dict={X: X_train, Y: Y_train})
                        #                     # TODO: h_attractor_collection reshapeing masking.
                        #                     h_attractor_val = None
                        #                     h_clean_val = batch_tensor_collect(sess, [h_clean_seq_flat],
                        #                                                                         X, Y, X_train, Y_train, BATCH_SIZE)[0]
                        #                     MIS.update(ploss, aloss, train_acc, test_acc, np.tanh(hid_vals), h_attractor_val, h_clean_val)

                        if (val_acc > best_val_acc):
                            best_train_acc = train_acc
                            best_test_acc = test_acc
                            best_att_loss = aloss
                            best_epoch = epoch
                            best_val_acc = val_acc

                            best_val_loss = ploss_val
                            best_train_loss = ploss
                            if ops['save_best_model']:
                                save_path = saver.save(sess, MODEL_DIRECTORY)
                            best_entropy = entropy
                        if (1.0 - 1e-15 < 0.0):
                            print('reached_peak')
                            break

                    if epoch > 1 and LOSS_SWITCH_FREQ > 0 \
                            and (epoch - 1) % LOSS_SWITCH_FREQ == 0:
                        train_prediction_loss = not train_prediction_loss

                    # MODEL TRAINING
                    batches = get_batches(BATCH_SIZE, X_train, Y_train)
                    for (batch_x, batch_y) in batches:
                        if (LOSS_SWITCH_FREQ == 0 or train_prediction_loss):
                            # Optimize all parameters except for attractor weights
                            _ = sess.run([pred_train_op],
                                         feed_dict={X: batch_x, Y: batch_y})
                        # Attractor:
                        if (N_ATTRACTOR_STEPS > 0):
                            batch_hid_vals = sess.run([A['h_net_seq_flat'] for att_name, A in G_attractors.items()],
                                                      feed_dict={X:batch_x,  Y:batch_y})

                            for i, attractor_name in enumerate(G_attractors.keys()):
                                A = G_attractors[attractor_name]
                                _ = sess.run(A['attr_train_op'], feed_dict={attractor_tgt_net: batch_hid_vals[i]})
                print("Optimization Finished!")

                if (REPORT_BEST_TRAIN_PERFORMANCE):
                    saved_train_acc.append(best_train_acc)
                    saved_test_acc.append(best_test_acc)
                    saved_att_loss.append(best_att_loss)
                    saved_entropy_final.append(best_entropy)
                    saved_epoch.append(best_epoch)

                    saved_val_acc.append(best_val_acc)
                    saved_val_loss.append(best_val_loss)
                    saved_traini_loss.append(best_train_loss)
                else:
                    saved_train_acc.append(train_acc)
                    saved_test_acc.append(test_acc)
                    #             saved_att_loss.append(aloss)

            save_results(ops, saved_epoch, saved_train_acc, saved_test_acc, saved_att_loss, saved_entropy_final, saved_val_acc,
                 saved_val_loss, saved_traini_loss, N_TRAIN, N_TEST, SEQ_LEN, comment=COMMENT)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
L2 reg-n
********** replication  0  **********
msnbc
(3689, 40) (3689, 40) (737, 40) (737, 40)
Logged Successfully: 

    model_type: 		GRU bidir(False), task: msnbc
    hid: 			50,
    h_hid: 			100
    n_attractor_iterations: 	15,
    attractor_dynamics: 	projection2
    attractor_noise_level: 	0.5
    attractor_noise_type: 	bernoilli
    attractor_regu-n: 		l2_regularization(lambda:0.0)
    word_embedding: size	(100), train(False)
    dropout: 			0.2
    TRAIN/TEST_SIZE: 	3689/0, SEQ_LEN: 40
Logged Successfully: 
0 10000000000.0 2.8903298
Logged Successfully: 
2018-05-17 18:59:54epoch=0; Loss Pred=2.8901; Val Loss=2.8903; Val Acc=0.0724; Loss Att={'forw': '1.0264'}; Train Acc=0.072; Test Acc=0.0707; Entropy={'forw': '7.0835'}; Entropy_Test=

0 2.8903298 2.7956798
Logged Successfully: 
2018-05-17 18:59:56epoch=1; Loss Pred=2.7979; Val Loss=2.7957; Val Acc=0.0974; Loss Att={'forw': '1.0168'}; Train

Logged Successfully: 
2018-05-17 19:00:47epoch=33; Loss Pred=1.9218; Val Loss=1.9081; Val Acc=0.4875; Loss Att={'forw': '0.7977'}; Train Acc=0.480; Test Acc=0.4743; Entropy={'forw': '7.9465'}; Entropy_Test=

0 1.9080912 1.9108592
Logged Successfully: 
2018-05-17 19:00:49epoch=34; Loss Pred=1.9272; Val Loss=1.9109; Val Acc=0.4925; Loss Att={'forw': '0.7889'}; Train Acc=0.484; Test Acc=0.4755; Entropy={'forw': '7.9289'}; Entropy_Test=

1 1.9080912 1.8530815
Logged Successfully: 
2018-05-17 19:00:51epoch=35; Loss Pred=1.8686; Val Loss=1.8531; Val Acc=0.5290; Loss Att={'forw': '0.7830'}; Train Acc=0.526; Test Acc=0.5180; Entropy={'forw': '8.1270'}; Entropy_Test=

0 1.8530815 1.8574303
Logged Successfully: 
2018-05-17 19:00:52epoch=36; Loss Pred=1.8696; Val Loss=1.8574; Val Acc=0.5345; Loss Att={'forw': '0.7737'}; Train Acc=0.532; Test Acc=0.5239; Entropy={'forw': '8.1343'}; Entropy_Test=

1 1.8530815 1.7968547
Logged Successfully: 
2018-05-17 19:00:54epoch=37; Loss Pred=1.8075; Val Loss=1.

0 1.4031008 1.3969809
Logged Successfully: 
2018-05-17 19:01:45epoch=69; Loss Pred=1.3987; Val Loss=1.3970; Val Acc=0.6407; Loss Att={'forw': '0.4698'}; Train Acc=0.637; Test Acc=0.6350; Entropy={'forw': '8.3148'}; Entropy_Test=

0 1.3969809 1.398012
Logged Successfully: 
2018-05-17 19:01:47epoch=70; Loss Pred=1.4005; Val Loss=1.3980; Val Acc=0.6399; Loss Att={'forw': '0.4614'}; Train Acc=0.638; Test Acc=0.6337; Entropy={'forw': '8.2819'}; Entropy_Test=

1 1.3969809 1.3826414
Logged Successfully: 
2018-05-17 19:01:48epoch=71; Loss Pred=1.3906; Val Loss=1.3826; Val Acc=0.6418; Loss Att={'forw': '0.4525'}; Train Acc=0.640; Test Acc=0.6355; Entropy={'forw': '8.2980'}; Entropy_Test=

0 1.3826414 1.3904243
Logged Successfully: 
2018-05-17 19:01:50epoch=72; Loss Pred=1.3929; Val Loss=1.3904; Val Acc=0.6424; Loss Att={'forw': '0.4444'}; Train Acc=0.640; Test Acc=0.6359; Entropy={'forw': '8.2929'}; Entropy_Test=

1 1.3826414 1.3803707
Logged Successfully: 
2018-05-17 19:01:51epoch=73; Loss Pre

0 1.3023099 1.2978954
Logged Successfully: 
2018-05-17 19:02:42epoch=105; Loss Pred=1.3006; Val Loss=1.2979; Val Acc=0.6567; Loss Att={'forw': '0.2552'}; Train Acc=0.657; Test Acc=0.6545; Entropy={'forw': '7.8425'}; Entropy_Test=

0 1.2978954 1.3003764
Logged Successfully: 
2018-05-17 19:02:44epoch=106; Loss Pred=1.3011; Val Loss=1.3004; Val Acc=0.6554; Loss Att={'forw': '0.2519'}; Train Acc=0.657; Test Acc=0.6515; Entropy={'forw': '7.8256'}; Entropy_Test=

1 1.2978954 1.2971768
Logged Successfully: 
2018-05-17 19:02:46epoch=107; Loss Pred=1.2971; Val Loss=1.2972; Val Acc=0.6569; Loss Att={'forw': '0.2493'}; Train Acc=0.658; Test Acc=0.6491; Entropy={'forw': '7.7605'}; Entropy_Test=

0 1.2971768 1.2975742
Logged Successfully: 
2018-05-17 19:02:47epoch=108; Loss Pred=1.2960; Val Loss=1.2976; Val Acc=0.6569; Loss Att={'forw': '0.2467'}; Train Acc=0.656; Test Acc=0.6504; Entropy={'forw': '7.7305'}; Entropy_Test=

1 1.2971768 1.2951092
Logged Successfully: 
2018-05-17 19:02:49epoch=109; Lo

1 1.2620379 1.2584991
Logged Successfully: 
2018-05-17 19:03:40epoch=141; Loss Pred=1.2573; Val Loss=1.2585; Val Acc=0.6644; Loss Att={'forw': '0.2120'}; Train Acc=0.667; Test Acc=0.6639; Entropy={'forw': '7.7114'}; Entropy_Test=

0 1.2584991 1.261543
Logged Successfully: 
2018-05-17 19:03:41epoch=142; Loss Pred=1.2599; Val Loss=1.2615; Val Acc=0.6643; Loss Att={'forw': '0.2119'}; Train Acc=0.665; Test Acc=0.6629; Entropy={'forw': '7.7078'}; Entropy_Test=

1 1.2584991 1.2576172
Logged Successfully: 
2018-05-17 19:03:43epoch=143; Loss Pred=1.2570; Val Loss=1.2576; Val Acc=0.6657; Loss Att={'forw': '0.2117'}; Train Acc=0.668; Test Acc=0.6645; Entropy={'forw': '7.6970'}; Entropy_Test=

0 1.2576172 1.2579716
Logged Successfully: 
2018-05-17 19:03:45epoch=144; Loss Pred=1.2566; Val Loss=1.2580; Val Acc=0.6659; Loss Att={'forw': '0.2116'}; Train Acc=0.667; Test Acc=0.6629; Entropy={'forw': '7.7081'}; Entropy_Test=

1 1.2576172 1.2611715
Logged Successfully: 
2018-05-17 19:03:46epoch=145; Los

...

0 2.4140093 2.4121416
Logged Successfully: 
2018-05-17 19:23:45epoch=12; Loss Pred=2.4269; Val Loss=2.4121; Val Acc=0.2729; Loss Att={'forw': '1.0000'}; Train Acc=0.287; Test Acc=0.2806; Entropy={'forw': '7.8468'}; Entropy_Test=

0 2.4121416 2.317695
Logged Successfully: 
2018-05-17 19:23:47epoch=13; Loss Pred=2.3352; Val Loss=2.3177; Val Acc=0.2566; Loss Att={'forw': '1.0000'}; Train Acc=0.276; Test Acc=0.2697; Entropy={'forw': '8.3363'}; Entropy_Test=

0 2.317695 2.3175666
Logged Successfully: 
2018-05-17 19:23:48epoch=14; Loss Pred=2.3352; Val Loss=2.3176; Val Acc=0.2599; Loss Att={'forw': '1.0000'}; Train Acc=0.276; Test Acc=0.2695; Entropy={'forw': '8.3363'}; Entropy_Test=

0 2.3175666 2.2711031
Logged Successfully: 
2018-05-17 19:23:49epoch=15; Loss Pred=2.2899; Val Loss=2.2711; Val Acc=0.2508; Loss Att={'forw': '1.0000'}; Train Acc=0.274; Test Acc=0.2681; Entropy={'forw': '8.6055'}; Entropy_Test=

0 2.2711031 2.2741215
Logged Successfully: 
2018-05-17 19:23:50epoch=16; Loss Pred

0 1.4474546 1.4536229
Logged Successfully: 
2018-05-17 19:24:30epoch=48; Loss Pred=1.4662; Val Loss=1.4536; Val Acc=0.6224; Loss Att={'forw': '1.0000'}; Train Acc=0.625; Test Acc=0.6209; Entropy={'forw': '8.8591'}; Entropy_Test=

1 1.4474546 1.419526
Logged Successfully: 
2018-05-17 19:24:31epoch=49; Loss Pred=1.4404; Val Loss=1.4195; Val Acc=0.6315; Loss Att={'forw': '1.0000'}; Train Acc=0.632; Test Acc=0.6277; Entropy={'forw': '8.9071'}; Entropy_Test=

0 1.419526 1.4117947
Logged Successfully: 
2018-05-17 19:24:33epoch=50; Loss Pred=1.4403; Val Loss=1.4118; Val Acc=0.6280; Loss Att={'forw': '1.0000'}; Train Acc=0.631; Test Acc=0.6279; Entropy={'forw': '8.9071'}; Entropy_Test=

0 1.4117947 1.3893272
Logged Successfully: 
2018-05-17 19:24:34epoch=51; Loss Pred=1.4197; Val Loss=1.3893; Val Acc=0.6377; Loss Att={'forw': '1.0000'}; Train Acc=0.637; Test Acc=0.6337; Entropy={'forw': '8.8179'}; Entropy_Test=

0 1.3893272 1.3965511
Logged Successfully: 
2018-05-17 19:24:35epoch=52; Loss Pred

3 1.256437 1.2559024
Logged Successfully: 
2018-05-17 19:25:15epoch=84; Loss Pred=1.2741; Val Loss=1.2559; Val Acc=0.6783; Loss Att={'forw': '1.0000'}; Train Acc=0.665; Test Acc=0.6632; Entropy={'forw': '8.3715'}; Entropy_Test=

0 1.2559024 1.2574518
Logged Successfully: 
2018-05-17 19:25:17epoch=85; Loss Pred=1.2694; Val Loss=1.2575; Val Acc=0.6767; Loss Att={'forw': '1.0000'}; Train Acc=0.665; Test Acc=0.6661; Entropy={'forw': '8.4042'}; Entropy_Test=

1 1.2559024 1.2515104
Logged Successfully: 
2018-05-17 19:25:18epoch=86; Loss Pred=1.2707; Val Loss=1.2515; Val Acc=0.6796; Loss Att={'forw': '1.0000'}; Train Acc=0.665; Test Acc=0.6631; Entropy={'forw': '8.4042'}; Entropy_Test=

0 1.2515104 1.253444
Logged Successfully: 
2018-05-17 19:25:19epoch=87; Loss Pred=1.2665; Val Loss=1.2534; Val Acc=0.6762; Loss Att={'forw': '1.0000'}; Train Acc=0.666; Test Acc=0.6629; Entropy={'forw': '8.4277'}; Entropy_Test=

1 1.2515104 1.2549784
Logged Successfully: 
2018-05-17 19:25:20epoch=88; Loss Pred

4 1.1965779 1.2080276
Logged Successfully: 
2018-05-17 19:26:00epoch=120; Loss Pred=1.2257; Val Loss=1.2080; Val Acc=0.6861; Loss Att={'forw': '1.0000'}; Train Acc=0.669; Test Acc=0.6678; Entropy={'forw': '8.8211'}; Entropy_Test=

5 1.1965779 1.2096974
Logged Successfully: 
2018-05-17 19:26:01epoch=121; Loss Pred=1.2238; Val Loss=1.2097; Val Acc=0.6859; Loss Att={'forw': '1.0000'}; Train Acc=0.669; Test Acc=0.6644; Entropy={'forw': '8.8654'}; Entropy_Test=

6 1.1965779 1.1913226
Logged Successfully: 
2018-05-17 19:26:02epoch=122; Loss Pred=1.2228; Val Loss=1.1913; Val Acc=0.6827; Loss Att={'forw': '1.0000'}; Train Acc=0.669; Test Acc=0.6675; Entropy={'forw': '8.8654'}; Entropy_Test=

0 1.1913226 1.196545
Logged Successfully: 
2018-05-17 19:26:04epoch=123; Loss Pred=1.2211; Val Loss=1.1965; Val Acc=0.6854; Loss Att={'forw': '1.0000'}; Train Acc=0.670; Test Acc=0.6656; Entropy={'forw': '8.9132'}; Entropy_Test=

1 1.1913226 1.2027333
Logged Successfully: 
2018-05-17 19:26:05epoch=124; Los

2 1.1677967 1.1733073
Logged Successfully: 
2018-05-17 19:26:42epoch=156; Loss Pred=1.1995; Val Loss=1.1733; Val Acc=0.6847; Loss Att={'forw': '1.0000'}; Train Acc=0.670; Test Acc=0.6679; Entropy={'forw': '9.2709'}; Entropy_Test=

3 1.1677967 1.1613163
Logged Successfully: 
2018-05-17 19:26:44epoch=157; Loss Pred=1.1991; Val Loss=1.1613; Val Acc=0.6858; Loss Att={'forw': '1.0000'}; Train Acc=0.670; Test Acc=0.6681; Entropy={'forw': '9.3022'}; Entropy_Test=

0 1.1613163 1.184357
Logged Successfully: 
2018-05-17 19:26:45epoch=158; Loss Pred=1.1987; Val Loss=1.1844; Val Acc=0.6840; Loss Att={'forw': '1.0000'}; Train Acc=0.671; Test Acc=0.6658; Entropy={'forw': '9.3022'}; Entropy_Test=

1 1.1613163 1.1782836
Logged Successfully: 
2018-05-17 19:26:46epoch=159; Loss Pred=1.1986; Val Loss=1.1783; Val Acc=0.6882; Loss Att={'forw': '1.0000'}; Train Acc=0.671; Test Acc=0.6691; Entropy={'forw': '9.3510'}; Entropy_Test=

2 1.1613163 1.1694119
Logged Successfully: 
2018-05-17 19:26:47epoch=160; Los